# REALTIME MIDI WITH RTMIDI

An explanation of how to send and receive real time midi data to an external app or device using the [python-rtmidi package](https://pypi.org/project/python-rtmidi/). Since two separate packages are involved (musx and rtmidi) this notebook uses full module names make it clear which features belong to which package.

<hr style="height:1px;color:gray">

Notebook imports:

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import threading, time
import rtmidi
import musx
print(f"musx version: {musx.version}")

## Setup

### Installing the python-rtmidi package

The python-rtmidi package provides support for reading and writing MIDI messages to and from MIDI ports. The package is based on the rtmidi C++ project, which is itself a sub project of a larger audio framework called [STK](https://github.com/thestk/stk) (Synthesis Tool Kit) developed by Prof. Gary Scavone at Stanford and McGill. 

To install the python-rtmidi package do:

```
$ python -m pip install python-rtmidi
```

<!-- When pip installs rtmidi it will attempt to compile C++ sources as part of the process so make sure you have a C++ compiler installed on your computer. If you are on a Mac with Xcode the python-rtmidi package should install without problems. On Windows, you may need to [follow these directions](https://spotlightkid.github.io/python-rtmidi/install-windows.html) to get the C++ code to build. You can also install rtmidi using the Anaconda package. -->

Some useful rt-midi links:

* [python-rtmidi installation instructions](https://spotlightkid.github.io/python-rtmidi/installation.html)
* [The rtmidi python documentation](https://spotlightkid.github.io/python-rtmidi/)
* [The rtmidi homepage and tutorial](http://www.music.mcgill.ca/~gary/rtmidi/)
* [STK](https://github.com/thestk)

### Listening to realtime playback

When you send MIDI data in real time you will need a MIDI sound module to receive and perform it.  There are a many possible options, here are a few examples:

* Connect a physical MIDI synthesizer to your computer via a USB cable.
* Use a DAW or commercial app to play MIDI via internal plugins: <!-- There are many possibilities, here are a few easy ones: -->
    - [Garage Band](https://www.apple.com/mac/garageband/) is available for free on the Mac. 
    - The [REAPER](https://www.reaper.fm/) DAW works well, runs everywhere, and has an unlimited trial period.
    - The [DLS-MIDI-synth](https://apps.apple.com/us/app/dls-midi-synth/id888643548?mt=12) app is an inexpensive wrapper around Apple's high-quality DLS synth. 

If you are on a Mac you should [enable Apple's IAC Driver bus](https://support.apple.com/guide/audio-midi-setup/transfer-midi-information-between-apps-ams1013/mac#:~:text=In%20the%20Audio%20MIDI%20Setup,to%20turn%20on%20the%20driver) to simplify routing data to/from different apps.

<!-- * [The c++ rtmidi codebase](https://github.com/thestk/rtmidi) -->

### Installing REAPER for realtime playback

1) Download and install [REAPER](https://www.reaper.fm/).

2) Launch REAPER and open its Preferences window (Command-,).

3) In the Preferences panel, open the Audio section and click on the MIDI Devices tab.

4) **ON THE MAC:** In the MIDI hardware panel that opens on the right side of the window, double click the line "Apple Inc. -IAC Driver - Bus 1", then click the "Enable input from this device" toggle button and select OK. (If you want to add a MIDI keyboard controller as an additional additional device you follow the exact same procedure again.)

5) If you are not on a Mac then add whatever MIDI input port is available and remember to open that port in python as your output port.

6) Back in the main REAPER window, use the app's menu bar and select the Track>Insert virtual instrument on new track... command.

7) **ON THE MAC:** In the dialog window that opens, type DLS in the textbox at the top of the window -- you will then see a line "AUi:DLSMusicDevice (Apple)"  in the list box on the right of the dialog. Click the Add button in the dialog to add the DLS device to your track.  

8) If you are not on a Mac, use whatever synth you have available. For purposes of this course it would be best if the synth were multitimbral, e.g. a sampler, or orchestral synth.

9) After you click Add in (step 7) you may see the synth's editor window pop up. Don't mess with this,  just close it by clicking on the window's close button.

10) Reaper should now be able to play any MIDI messages you send to it on the IAC Driver Bus or whatever input port you opened.

## MIDI messages and MIDI ports

A MIDI message is a sequence of bytes that conform to the [MIDI message standard](https://www.midi.org/specifications) as defined by the [MIDI Association](https://www.midi.org/). In the Python environment MIDI messages can be represented in  different ways: as a list or tuple of bytes, as byte arrays, or as objects defined by a MIDI class taxonomy.

A MIDI port is an input or output connection for sending/receiving MIDI messages to/from external MIDI hardware or software.  MIDI ports and devices were originally implemented in hardware but are now overwhelmingly software based.

### The musx.midi subpackage 

The `musx.midi` subpackage provides several layers of support for reading and writing MIDI messages to MIDI files and ports. For convenience, symbols defined in these sub packages are automatically imported and made available directly from the parent musx package. In this notebook we will use the "low level" musx.midi.midimsg module to send/receive data to/from the rtmidi system.

### The  musx.midi.midimsg submodule

THe midimsg module provides constants and constructor functions for manipulating low-level *byte* messages as described in the MIDI specification. At this module's level, a MIDI message is just Python list containing one or more bytes of integer data. The complete list of constructors is documented in the [musx.midi.midimsg module](https://musx-admin.github.io/musx/midi/midimsg.html).



Examples of three MIDI byte messages. Note: To avoid package confusion, this notebook includes the parent package names when calling midi functions provided by musx and rtmidi:

In [ ]:
print(f"midi note on: {musx.note_on(4, 60, 90)}")
print(f"midi note off: {musx.note_off(4, 60, 127)}")
print(f"midi program change: {musx.program_change(0, 33)}")

### The musx.midi.gm submodule

The midi.gm submodule defines symbolic names for all the General MIDI constants. Since there are hundreds of these in the General Midi spec, the midi.gm submodule is _not_ auto-imported into musx. Instead, you can import these gm instruments, control changes, and percussion map constants like this:

In [ ]:
from musx.midi.gm import OrchestralHarp, ModulationWheel_LSB, SplashCymbal

print(f"midi note on: {musx.note_on(9, SplashCymbal, 90)}")
print(f"midi program change: {musx.program_change(0, OrchestralHarp)}")
print(f"midi control change: {musx.control_change(3, ModulationWheel_LSB, 66)}")

## Sending MIDI messages using rtmidi
<!-- Working with the python-rtmidi package --?

### Opening an output port

To open a MIDI output port in rtmidi you first create a `MidiOut` object and then open one of its output ports. To see what ports are available call `rtmidi.MidiOut.get_ports()`, which will return a list of all the available port names.  Note that -- unlike REAPER's port preferences -- the list that rtmidi returns will only contain the ports currently registered in your host at the time the function is called. Similarly, it is possible for ports in this list to 'disappear' asynchronously, for example, by someone unplugging a midi interface from the host or quitting an app that had opened a midi port.

Accessing rtmidi's MidiOut object:

In [ ]:
midiout = rtmidi.MidiOut()
print(midiout)

Once the midiout object is allocated you can retrieve the list of available (open) output ports currently active on your computer:

In [ ]:
outports = midiout.get_ports()
print("available ports:", outports)

To open a particular output port you pass its *index* in the outports list to rtmidi's `MidiOut.open_port()` method.

Tip: Do not depend on this index always being the same! To make it obvious in your code which port you are opening, consider passing the port's name to `list.index()` to return the port's index number from its name. That way (1) you will explicitly see which port you are trying to working with, (2) the correct port open will even if it's index changes, and (3) if the port isn't available the error will be reported before attempting to open it:

This example assumes you have enabled the IAC Driver Bus on a Mac.  If this is not the case then pick a port from the list that was returned in the previous example:

In [ ]:
midiout.open_port(outports.index('IAC Driver Bus 1'))

At this point you have a open output port. You can always check to see if a port is open using the is_port_open() function. (If you do not receive a True value from this  example, check to make sure that your midi "device" is running and has an open input port and use that port as your output port.)

In [ ]:
midiout.is_port_open()

### Sending midi messages

Once a MIDI output port is opened MIDI messages can be sent to it. One nice feature of the rtmidi library (well....at least for programmers ;) ) is that its midi messages are just iterables of MIDI bytes, i.e. its message interface doesn't involve class instances but instead adheres exactly to the byte descriptions of the MIDI protocol for maximum flexibility.  

`MidiOut.send_message(message)`

Use rtmidi's MidiOut.send_message() to send messages to the output port.

This example allocates a midi NoteOn and sends it to the open output port. You should hear a sound -- if you don't then check to see that your output port is valid, and is active as the input port in your DAW.

In [ ]:
msg = musx.note_on(0, 60, 80)
print(f"sending note on message: {msg}")
midiout.send_message(msg)

Once a NoteOn is sent be sure to send a NoteOff, even if you didn't hear anything:

In [ ]:
msg = musx.note_off(0, 60, 127)
print(f"sending note off message: {msg}")
midiout.send_message(msg)

If you send a NoteOn immediately followed by a NoteOff you won't hear much because the off message will immediately cancel the on message:

In [ ]:
on = musx.note_on(0, 60, 80)
print(f"sending note on message: {on}")
midiout.send_message(on)
off = musx.note_off(0, 60, 127)
print(f"sending note off message: {off}")
midiout.send_message(off)

To play a note on and off in *real time* we need to wait for some amount of time between the on and off events; in music this kind of value is called *duration*.  We can use Python's `time.sleep()` function to pause between the two sends for the given number of seconds before processing continues:

In [ ]:
print("I am awake...")
print("Time to take a nap...")
time.sleep(3)
print("...nap time is over!")

This example uses time() to play on and off pairs in a loop, each iteration performs a random key and instrument pair and sound for a random amount of time. 

In [ ]:
def getinsname(n):
    return musx.midi.gm.instrument_names[n]

for i in range(10):
    # pick a random midi instrument (0-127)
    ins = musx.between(0, 128)
    # send a program change to select the instrument on channel 0
    midiout.send_message(musx.program_change(0, ins))
    # pick a random midi key number
    key = musx.between(40, 80)
    # pick a random duration
    dur = musx.pick(.5, 1, 1.5)
    # send it out
    print(f"playing '{getinsname(ins)}' key: {key}, dur: {dur}")
    midiout.send_message(musx.note_on(0, key, 80))
    # wait for duration
    time.sleep(dur)
    # stop the note
    midiout.send_message(musx.note_off(0, key, 127))

print("All done!")
# set your midi synth back to piano
midiout.send_message(musx.program_change(0, 0))

## Concurrent, realtime performance using Python threads

Python threads run code in separate lines of execution, in parallel with the python interpreter (main thread) or other concurrently running threads. 

Think of running threads in parallel like a musical performance:  all the musicians (threads) play their parts concurrently and in real time, each with their own unique set of notes and rhythms. 

Here is an example of two threads performing a second species counterpoint in real time:

In [ ]:
def cantusfirmus(notes):
    for key in musx.keynum(notes):
        print("-------------------\ncantus:", key)
        midiout.send_message(musx.note_on(0, key, 80))
        time.sleep(1)
        midiout.send_message(musx.note_off(0, key, 127))

def counterpoint(notes):
    for key in musx.keynum(notes):
        print("melody:", key)
        midiout.send_message(musx.note_on(0, key, 80))
        time.sleep(.5)
        midiout.send_message(musx.note_off(0, key, 127))
    
cantus = threading.Thread(target=cantusfirmus, args=("c4 d e f g e d c",))   
melody = threading.Thread(target=counterpoint, args=("c5 a4 b a g c5 d c e d c b4 a b c5",)) 
cantus.start()
melody.start()

### Caveat

Before you get too excited...Python threads  mimic multiprocessing, they are not really separate processes running on separate processors! They also can't be counted on for extremely precise timing, which means that if you use two threads their timed events may "drift" over time. 

Despite these caveats, threads are still quite useful for implementing cooperative tasks and even real time musical performances, in particular if the performance involves playback of one musx Seq (sequence) that itself contains multiple performance timelines. In this case, there will be no "drift" since all the parts are being performed in just one thread. The Score.compose() method is quite fast so


## Playing sequences in real time

The following section describes an implementation of a midi player that will perform a musx.Seq (sequence) of Notes in "real time" to an external midi device. 

`play_rtmidi(seq,  port,  block=True)`

Performs a sequence of Notes or MidiEvents in real time. If the block argument to midi_play() is true then the current thread of execution (the thread in which midi_play() is launched) is "blocked" i.e. unable to continue its work until the child thread has completed. Blocking is not needed if you are working inside the python interpreter, but if the score is to be played inside a python script blocking is crucial: without blocking the script will start the thread but not wait for it to complete and so very likely exit from script well before the performance thread would be able to generate its sound.

In [ ]:
import sys
from musx.midi.midievent import MidiEvent
from musx.midi.midimsg import note_on, note_off

def play_rtmidi(seq, port, block=True):
    """
    Plays a Note or MidiEvent sequence in real time out an open rtmidi output port.
      
    Parameters
    ----------
    seq : Seq
        A sequence of Note or MidiEvent objects.
    port : rtmidi.MidiOut
        An open rtmidi MidiOut object.
    block : bool
        If true then midi_play() will block for the duration of the playback.
    """
    
    def _thread_player(midi, outport):
        '''Runs inside a thread to send midi events in real time to an open output port.'''
        length = len(midi)
        # get time of next message
        nexttime = midi[0].time
        thistime = nexttime
        i = 0
        while i < length:
            if midi[i].time == thistime:
                #print(f'playing {seq[i]}')
                outport.send_message(midi[i].message)
                i += 1
                continue
            # if here then sleep because midi[i] is later than thistime
            nexttime = midi[i].time
            #print(f'waiting {nexttime-thistime}')
            time.sleep(nexttime - thistime) 
            thistime = nexttime
            
    if not seq.events:
        raise ValueError(f"no midi events to play")
    if not 'rtmidi' in sys.modules:
        raise RuntimeError(f"module rtmidi is not loaded")
    if not isinstance(port, sys.modules['rtmidi'].MidiOut):
        raise TypeError(f"port is not an instance of rtmidi.MidiOut")
    if isinstance(seq[0], musx.Note):
        seq = _notes_to_midi(seq)
    # create a thread and pass it _midi_player.
    player = threading.Thread(target=_thread_player, args=(seq.events, port))#, daemon=True
    # start the playback
    player.start()
    if block:
        # wait until playback is complete before returning from function
        player.join()

print(f"play_rtmidi: {play_rtmidi}")

This example composes a two part score and plays it in real time to an open output port. Blocking is set so you will not see the "All done!" message until the thread has finished.

In [ ]:
score = musx.Score(out=musx.Seq())

def species(score, notes, rhy):
    for key in musx.keynum(notes):
        note = musx.Note(time=score.now, duration=rhy, pitch=key)
        score.add(note)
        yield rhy
        
cantus = "c4 d e f g e d c"
melody = "c5 a4 b a g c5 d c e d c b4 a b c5"
score.compose([species(score, cantus, 1), species(score, melody, .5)])

print("Playing score in real time...")
play_rtmidi(score.out, midiout, block=True)
print("All done!")

If block is False then the current thread of execution will not be blocked so the code continues execution while the score is being performed.

In this unblocked example we see the "all done!" messages before the performance is complete.

In [ ]:
play_rtmidi(score.out, midiout, block=False)

for x in range(10): 
    print(f"all done {x}!")

<!-- Playing MIDI sequences in real time
MidiSeq.play(midiout, block=True)

The MidiSeq.play() method sends all the sequences MIDI messages in real time, but using a separate thread of execution to perform in. The midiout parameter is the open rtmidi output object. The block parameter determines if the current thread of execcution is blocked while the playback executes. If you are calling MidiSeq.play() inside the "main" block of a python script then you will want to set block to True or else your script will exit before the playback thread has a chance to start its playback! On the other hand, if you are calling MidiSeq.play() interactively in the interpreter you will probably not want to block so that the interpreter can still evaluate statements while the playback is running.

In this example the player is launched without blocking the main thread so while it plays music the interpreter thread is still able to evaluate expressions. -->

## Realtime MIDI input

The rtmidi package provides MIDI input ports to receive data from external devices and apps.  The way you initialize midi input ports in rtmidi as almost identical to opening output ports: you access a list of available input ports and then open the port of interest. Note that only one input port can be open at a time.

Important: you must to start or connect your input device of choice before you evaluate the next cell to create the MidiIn object.  If you dont have a MIDI keyboard to hook up, you can install and launch the free [Virtual MIDI Piano Keyboard](https://sourceforge.net/projects/vmpk/). To start the vpmk app for the first time on a Mac, control-click the app icon  and select Open to agree to open an app from an 'unidentified developer'. Once the app is running click on Edit&gt;MIDI connections and set the dialog controls to this:

Once your keyboard device/app is connected/running, try these examples. If you are using a physical keyboard make sure it does not send active sensing messages.

Create the MidiIn object:

In [ ]:
midiin = rtmidi.MidiIn()
print(midiin)

Access the list of available input ports:

In [ ]:
inports = midiin.get_ports()
print("available input ports:", inports)

Now open the port of interest, in the example I am opening case the Virtual MIDI Piano Keyboard:

In [ ]:
midiin.open_port(inports.index('VMPK Output'))
#midiin.open_port(inports.index('IAC Driver Bus 1'))
#midiin.open_port(inports.index('MK-249C USB MIDI keyboard'))
#midiin.open_port(inports.index('MIDI Out'))

Similar to the midiout device, you can test if the midiin port is ready:

In [ ]:
midiin.is_port_open()

## Setting a midi input callback

Assigning a *callback* to the open input port will allow you to process incoming messages 
asynchronously, on demand, thereby removing any need to poll the port in the main thread or run loops in child threads. This is almost always preferable to synchronous "polling" in a loop.

Your callback function must accept two arguments *(message, data)* where *message* is a tuple containing the midi message and the lapsed time (see: get_message() above) and *data* is any optional data that you assigned to the input port.  The port does not use the data, but passes it to your callback each time it is called. For example, your data might be a structure that you want your callback to add messages to, or a function the callback should invoke every time the callback is triggered:

In [ ]:
def my_midi_callback(message, data):
    '''This is my callback function.'''
    print("my midi callback: message =", message, ", data =", data)

print(f"my_midi_callback: {my_midi_callback}")

Use `MidiIn.set_callback(func, data=None)` to set your callback function. You can also (optionally) pass
any data you want your callback to use as it processes incoming midi messages.

Evaluate this next cell and start playing your midi keyboard. Any incoming messages will be processed by your callback without blocking the main thread:

In [ ]:
midiin.set_callback(my_midi_callback)
print("Callback set!")

When you are done you can stop the callback from happening by calling cancel_callback():

In [ ]:
midiin.cancel_callback()
print("Callback cancelled!")